<a href="https://colab.research.google.com/github/bagustris/ravdess_song/blob/main/ravdess_song_sd_fc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAVDESS Song Emotion Recognition
Including feature extraction process, model: Dense  
to be run from Google Colab

In [1]:
import glob
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
!git clone https://github.com/bagustris/ravdess_song.git

Cloning into 'ravdess_song'...
remote: Enumerating objects: 1144, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 1144 (delta 47), reused 32 (delta 12), pack-reused 1057
Receiving objects: 100% (1144/1144), 232.30 MiB | 24.27 MiB/s, done.
Resolving deltas: 100% (51/51), done.
Checking out files: 100% (1030/1030), done.


In [3]:
cd ravdess_song/

/content/ravdess_song


In [4]:
# list all files in the dataset
data_path = 'archive'
files = glob.glob(os.path.join(data_path + '/*/', '*.wav'))
files.sort()

In [5]:
# check by showing the first file
files[0]

'archive/Actor_01/03-02-01-01-01-01-01.wav'

In [6]:
# function to extract feature
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name, sr=None)
    stft = np.abs(librosa.stft(X))
    mfcc = np.mean(librosa.feature.mfcc(
        y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    mfcc_std = np.std(librosa.feature.mfcc(
        y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(
        S=stft, sr=sample_rate).T, axis=0)
    chroma_std = np.std(librosa.feature.chroma_stft(
        S=stft, sr=sample_rate).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
    mel_std = np.std(librosa.feature.melspectrogram(
        X, sr=sample_rate).T, axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(
        S=stft, sr=sample_rate).T, axis=0)
    contrast_std = np.std(librosa.feature.spectral_contrast(
        S=stft, sr=sample_rate).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(
        y=librosa.effects.harmonic(X), sr=sample_rate).T, axis=0)
    tonnetz_std = np.std(librosa.feature.tonnetz(
        y=librosa.effects.harmonic(X), sr=sample_rate).T, axis=0)
    return (mfcc, chroma, mel, contrast, tonnetz,
            mfcc_std, chroma_std, mel_std, contrast_std, tonnetz_std)

In [7]:
# create empty list to store features and labels
feat = []
lab = []

In [8]:
# iterate over all files
for file in files:
    print("processing ...", file)
    feat_i = np.hstack(extract_feature(file))
    lab_i = os.path.basename(file).split('-')[2]
    feat.append(feat_i)
    lab.append(int(lab_i)-1)  # make labels start from 0

processing ... archive/Actor_01/03-02-01-01-01-01-01.wav
processing ... archive/Actor_01/03-02-01-01-01-02-01.wav
processing ... archive/Actor_01/03-02-01-01-02-01-01.wav
processing ... archive/Actor_01/03-02-01-01-02-02-01.wav
processing ... archive/Actor_01/03-02-02-01-01-01-01.wav
processing ... archive/Actor_01/03-02-02-01-01-02-01.wav
processing ... archive/Actor_01/03-02-02-01-02-01-01.wav
processing ... archive/Actor_01/03-02-02-01-02-02-01.wav
processing ... archive/Actor_01/03-02-02-02-01-01-01.wav
processing ... archive/Actor_01/03-02-02-02-01-02-01.wav
processing ... archive/Actor_01/03-02-02-02-02-01-01.wav
processing ... archive/Actor_01/03-02-02-02-02-02-01.wav
processing ... archive/Actor_01/03-02-03-01-01-01-01.wav
processing ... archive/Actor_01/03-02-03-01-01-02-01.wav
processing ... archive/Actor_01/03-02-03-01-02-01-01.wav
processing ... archive/Actor_01/03-02-03-01-02-02-01.wav
processing ... archive/Actor_01/03-02-03-02-01-01-01.wav
processing ... archive/Actor_01

In [9]:
len(feat)
len(lab)

1012

In [10]:
# assign hsf to X, lab to y; reshape X for LSTM
X = np.array(feat)
y = np.array(lab)

In [11]:
# split into train and test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [12]:
# functio to build FCN 
def model_dense():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.BatchNormalization(axis=-1,
              input_shape=(x_train.shape[1:])))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(6, activation='softmax'))

    # compile model: set loss, optimizer, metric
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model


In [13]:
# create the model
model = model_dense()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 386)               1544      
_________________________________________________________________
dense (Dense)                (None, 256)               99072     
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 1542      
Total params: 233,742
Trainable params: 232,970
Non-trainable params: 772
________________________________________________

In [14]:
# train the LSTM model
hist = model.fit(x_train, y_train, epochs=100, shuffle=True, batch_size=16)

Epoch 1/100
57/57 [==============================] - 1s 5ms/step - loss: 1.2263 - accuracy: 0.5143
Epoch 2/100
57/57 [==============================] - 0s 5ms/step - loss: 0.5951 - accuracy: 0.7692
Epoch 3/100
57/57 [==============================] - 0s 5ms/step - loss: 0.4456 - accuracy: 0.8330
Epoch 4/100
57/57 [==============================] - 0s 4ms/step - loss: 0.3530 - accuracy: 0.8582
Epoch 5/100
57/57 [==============================] - 0s 4ms/step - loss: 0.3254 - accuracy: 0.8835
Epoch 6/100
57/57 [==============================] - 0s 5ms/step - loss: 0.3105 - accuracy: 0.8934
Epoch 7/100
57/57 [==============================] - 0s 5ms/step - loss: 0.2139 - accuracy: 0.9209
Epoch 8/100
57/57 [==============================] - 0s 5ms/step - loss: 0.2473 - accuracy: 0.9077
Epoch 9/100
57/57 [==============================] - 0s 5ms/step - loss: 0.2000 - accuracy: 0.9319
Epoch 10/100
57/57 [==============================] - 0s 5ms/step - loss: 0.2370 - accuracy: 0.9187
Epoch 11/

In [15]:
evaluate = model.evaluate(x_test, y_test, batch_size=16)
print("Loss: ", evaluate[0], "--> Accuracy: ", evaluate[1])

7/7 [==============================] - 0s 3ms/step - loss: 0.4456 - accuracy: 0.9314
Loss:  0.4455648958683014 --> Accuracy:  0.9313725233078003
